In [1]:
from llm_core.assistants import OpenAIAssistant , Analyst
from dataclasses import dataclass
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import os 
from langchain_core.prompts import ChatPromptTemplate 
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [2]:
model = "gpt-3.5-turbo-0613"
assistant = OpenAIAssistant

@dataclass
class EventRequette :
  
    system_prompt = "You are a system assistant specialiszing in wrting / explaining "
    prompt = """ #Context 
    In our Api we record all the actions of different users with different informations 

    #TODO :
    Use the information contained in {events} to make a professional presentation of each event as accurately as possible, separated into points: one point per event.Each presentation must be written. 
    I want you to write in in the requested language : {lang}.
    Write the event presentation directly  """
    response: str

In [3]:
def test_py_llm_core_first(data):
    with OpenAIAssistant(EventRequette, model) as assistant: 
        response  = assistant.process(events = data, lang = "en")
    final_response = response.response
    print(final_response)

In [4]:
def test_py_llm_core_second(data):
    analyst = Analyst(model , assistant)
    query = """Use the informations provided in data to make a professional presentation of each event as accurately as possible and with as many ifnormations as possible, separated into points: one point per event. Each point must be written with natural sentences. 
    I want you to write in in the requested language : {lang}.
    """
    context  = {"context": "In our Api we record all the actions of different users with different informations",
                "lang": "en ",
                "data": data}
    response = analyst.ask(query, context)
    print(response.content)

In [5]:
events = [
        {
            "user": "",
            "user_group" : "iii",
            "event": "ButtonClicked",
            "created": datetime.now().isoformat(),
            "userinfo": "usersystem",
            "feature": "analyse_doc:ChooseDoc",
            "action_type": "CHOOSE"
        },
        {
            "user": "2",
            "user_group" : "iv",
            "event": "PageLoaded",
            "created": datetime.now().isoformat(),
            "userinfo": "usersystem",
            "feature": "analyse_doc:ViewDoc",
            "action_type": "ASKMEYA"
        },
        {
            "user": "1",
            "user_group" : "v",
            "event": "FormSubmitted",
            "created": datetime.now().isoformat(),
            "userinfo": "usersystem",
            "feature": "analyse_doc:SubmitDoc",
            "action_type": "CREATE"
        },
        {
            "user": "3",
            "user_group": "admin",
            "event": "ButtonClicked",
            "created": "2023-03-01T14:00:00Z",
            "userinfo": "John Doe",
            "feature": "search",
            "action_type": "NAVIGATE"
        },
        {
            "user": "3",
            "user_group": "user",
            "event": "DocumentModified",
            "created": "2023-03-01T13:00:00Z",
            "userinfo": "Jane Doe",
            "feature": "documents",
            "action_type": "MODIF"
        }
        
    ]

In [6]:
test_py_llm_core_first(events)

Event Presentations:

1. User 'iii' clicked a button on 'analyse_doc:ChooseDoc' feature at '2024-02-12T13:56:53.087786'.

2. User '2' from user group 'iv' loaded a page on 'analyse_doc:ViewDoc' feature at '2024-02-12T13:56:53.087797'.

3. User '1' from user group 'v' submitted a form on 'analyse_doc:SubmitDoc' feature at '2024-02-12T13:56:53.087799'.

4. User '3' from user group 'admin' clicked a button on 'search' feature at '2023-03-01T14:00:00Z'.

5. User '3' from user group 'user' modified a document on 'documents' feature at '2023-03-01T13:00:00Z'.


In [7]:
test_py_llm_core_second(events)

Sure! Here is a professional presentation of each event:

1. User group 'iii' clicked a button on 'analyse_doc:ChooseDoc' feature at '2024-02-12T13:56:53.087786'.

2. User '2' from user group 'iv' loaded a page on 'analyse_doc:ViewDoc' feature at '2024-02-12T13:56:53.087797'.

3. User '1' from user group 'v' submitted a form on 'analyse_doc:SubmitDoc' feature at '2024-02-12T13:56:53.087799'.

4. User '3' from user group 'admin' clicked a button on 'search' feature at '2023-03-01T14:00:00Z'.

5. User '3' from user group 'user' modified a document on 'documents' feature at '2023-03-01T13:00:00Z'.


In [8]:
def test_langchain(events):
    putpout_parser = StrOutputParser()
    prompt_template =  ChatPromptTemplate.from_template(
    """Use the informations provided in {data} to make a professional presentation of each event as accurately as possible and with as many ifnormations as possible, separated into points: one point per event. Each point must be written with natural sentences. 
        I want you to write in in the requested language  :{lang} .
        """)
    llm = ChatOpenAI(temperature=0.9, model_name="gpt-3.5-turbo")
    chain = prompt_template | llm | putpout_parser

    resp= chain.invoke({"data":events,"lang":"en"})
    print(resp)

In [9]:
test_langchain(events)

1. User group "iii" clicked a button on the "analyse_doc" feature named "ChooseDoc" at 13:56:53 on February 12, 2024. This action was performed by the system user "usersystem".

2. User group "iv" loaded a page on the "analyse_doc" feature named "ViewDoc" at 13:56:53 on February 12, 2024. This action was performed by the system user "usersystem".

3. User group "v" submitted a form on the "analyse_doc" feature named "SubmitDoc" at 13:56:53 on February 12, 2024. This action was performed by the system user "usersystem".

4. User with ID "3" and user group "admin" clicked a button on the "search" feature at 14:00:00 on March 1, 2023. This action was performed by the user named "John Doe".

5. User with ID "3" and user group "user" modified a document on the "documents" feature at 13:00:00 on March 1, 2023. This action was performed by the user named "Jane Doe".


In [10]:
def test_longchain_second(events):
    pout_parser = StrOutputParser()
    prompt_template = PromptTemplate(
        input_variables=["events","lang"],
        template = """Use the informations provided in {events} to make a professional presentation of each event as accurately as possible and with as many ifnormations as possible, separated into points: one point per event. Each point must be written with natural sentences. 
        I want you to write in in the requested language  :{lang} .
        """
    )
    llm = ChatOpenAI(temperature=0.9, model_name="gpt-3.5-turbo")
    chain = LLMChain(llm=llm, prompt=prompt_template, output_parser=pout_parser)
    result = chain.run(events=events, lang="fr")
    print(result)
    

In [11]:
test_longchain_second(events)

/home/dima/.virtualenvs/djangoEnv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Voici une présentation professionnelle de chaque événement, en utilisant les informations fournies :

1. Un utilisateur du groupe iii a cliqué sur un bouton pour choisir un document dans la fonctionnalité "analyse_doc:ChooseDoc". Cet événement s'est produit le 12 février 2024 à 13h56 et 53 secondes. L'utilisateur était identifié dans le système sous le nom "usersystem".

2. Un utilisateur du groupe iv a chargé une page dans la fonctionnalité "analyse_doc:ViewDoc". Cet événement s'est également produit le 12 février 2024 à 13h56 et 53 secondes. L'utilisateur était identifié sous le numéro 2 et son groupe était "iv". L'action était de demander quelque chose avec l'action_type "ASKMEYA".

3. Un utilisateur du groupe v a soumis un formulaire dans la fonctionnalité "analyse_doc:SubmitDoc". Cet événement a eu lieu le 12 février 2024 à 13h56 et 53 secondes. L'utilisateur était identifié sous le numéro 1 et son groupe était "v". L'action effectuée était de créer quelque chose avec l'action_typ